In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [2]:
#nltk.download()

In [3]:
Historical_data = pd.read_csv(r"C:\Users\prash\OneDrive\Documents\CTC\Python Files\Python Rahul Sir New/Historical labeled data.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\prash\\OneDrive\\Documents\\CTC\\Python Files\\Python Rahul Sir New/Historical labeled data.csv'

In [16]:
Historical_data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [4]:
Realtime_data = pd.read_excel(r'C:\Users\prash\OneDrive\Documents\CTC\Python Files\Python Rahul Sir New/Real time data.xlsx')

In [5]:
Realtime_data.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [18]:
Historical_data.shape

(7920, 3)

In [19]:
Realtime_data.shape

(1953, 2)

In [21]:
Historical_data.isnull().any()

id       False
label    False
tweet    False
dtype: bool

In [20]:
Realtime_data.isnull().sum()

id       0
tweet    0
dtype: int64

# Text Pre Processing

In [22]:
#URL removal
Historical_data['Cleaned_content'] = Historical_data['tweet'].str.replace('http\S+|www.\S+', '', case=False)

In [24]:
#URL removal
Realtime_data['Cleaned_content'] = Realtime_data['tweet'].str.replace('http\S+|www.\S+', '', case=False)

In [25]:
import re 
def preprocess_text(text):
    text = re.sub('((www\.[^\s]+)|-(https?://[^\s]+))',' ', text) #URLs
    text = re.sub('@[^\s]+',' ', text) #Name tags removal
    text = text.lower().replace("ё", "е") #Case conversion
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text) #ASC removal
    text = re.sub(r'\d+','', text) #Numbers
    text = re.sub(' +',' ', text) #Punctation removal
    return text.strip()

In [27]:
Historical_data['Cleaned_content'] = [preprocess_text(str(i)) for i in Historical_data['Cleaned_content']] 

In [29]:
Realtime_data['Cleaned_content'] = [preprocess_text(str(i)) for i in Realtime_data['Cleaned_content']] 

In [30]:
stop = set(stopwords.words('english'))

In [31]:
#Removing stop words
Historical_data['Cleaned_content'] = Historical_data['Cleaned_content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [32]:
#Removing stop words
Realtime_data['Cleaned_content'] = Realtime_data['Cleaned_content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# Classification

In [34]:
data_x = Historical_data[['Cleaned_content']].values.tolist()

data_y = np.asarray(Historical_data['label'],dtype=int)

X_train, X_test, y_train, y_test = model_selection.train_test_split(data_x, data_y, test_size=0.3)

In [35]:
cv =  CountVectorizer(lowercase=False,stop_words=stop, ngram_range=(1,3))

In [42]:
X_train_cv = cv.fit_transform(str(e) for e in X_train)
X_test_cv = cv.transform(str(i) for i in X_test)

In [43]:
X_train_cv.shape

(5544, 98960)

In [44]:
X_test_cv.shape

(2376, 98960)

In [45]:
#Naive Bayes Model
nbayes = MultinomialNB()

nbayes.fit(X_train_cv, y_train)

MultinomialNB()

In [46]:
y_predictions = nbayes.predict(X_test_cv)

In [47]:
y_predictions

array([0, 0, 0, ..., 1, 1, 1])

In [48]:
y_result = list(y_predictions)

In [49]:
print("Accuracy % :",metrics.accuracy_score(y_test, y_predictions)*100)
print("Precision Score: ", metrics.precision_score(y_test, y_predictions))
print("Recall Score: ",metrics.recall_score(y_test, y_predictions) )
print("F1 Score: ",metrics.f1_score(y_test, y_predictions) )

Accuracy % : 87.79461279461279
Precision Score:  0.7838345864661654
Recall Score:  0.7043918918918919
F1 Score:  0.7419928825622776


In [50]:
print(metrics.classification_report(y_test, y_predictions))

              precision    recall  f1-score   support

           0       0.91      0.94      0.92      1784
           1       0.78      0.70      0.74       592

    accuracy                           0.88      2376
   macro avg       0.84      0.82      0.83      2376
weighted avg       0.87      0.88      0.88      2376



In [ ]:
#Real time testing

In [51]:
Realtime_test = Realtime_data[['Cleaned_content']].values.tolist()

In [53]:
Realtime_test_cv = cv.transform(str(i) for i in Realtime_test)

In [55]:
Realtime_test_cv.shape

(1953, 98960)

In [56]:
Realtime_predictions = nbayes.predict(Realtime_test_cv)

In [58]:
data_testlist = list(Realtime_data["tweet"])

In [59]:
label_preds = list(Realtime_predictions)

In [60]:
output_df = pd.DataFrame({'comment':data_testlist ,'label':label_preds})
output_df

,comment,label
0,I hate the new #iphone upgrade. Won't let me d...,1
1,currently shitting my fucking pants. #apple #i...,1
2,"I'd like to puts some CD-ROMS on my iPad, is t...",1
3,My ipod is officially dead. I lost all my pict...,1
4,Been fighting iTunes all night! I only want th...,1
...,...,...
1948,"#SamsungGalaxyNote7 Explodes, Burns 6-Year-Old...",0
1949,Now Available - Hoodie. Check it out here - ht...,0
1950,There goes a crack right across the screen. If...,1
1951,@codeofinterest as i said #Adobe big time we m...,1


In [62]:
Realtime_data["Predicted_lable"] = output_df["label"]

In [63]:
Realtime_data.to_csv("Predicted_file.csv")